<a href="https://colab.research.google.com/github/bivekSapkota/Constraint_Programming-for-Nursing-Depart/blob/main/CSUP_nursing_basic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Ortools

In [ ]:
from ortools.sat.python import cp_model
model= cp_model.CpModel()

In [ ]:
# Code written by Bivek Sapkota

#Lets define decision variable and constants
no_of_weeks= 4
no_of_days= 6
no_of_blocks= 4 #no. of blocks per day, look at blocks_arrangement for per day arrangement
blocks_arrangement= [ # Maximum four blocks each day, this list can also be used to calculate the total blocks per day, just use sum(blocks_arrangement)[day]
                     [0,1,0,0], #Monday
                     [1,1,1,1], #Tuesday
                     [1,1,1,1], #Wednesday
                     [0,0,1,0], #Thursday
                     [1,1,1,1], #Friday
                     [1,1,1,1]  #Saturday
                    ]

block_availability= [# M, T, W, Th,F, S   Defines if block is available for each type of group for each day, 1 = available, 0= not available
                      [1, 0, 1, 0, 0, 1], #senior      {Mon, Wed, Sat}
                      [0, 1, 0, 1, 1, 1], #junior      {Tue, Wed, Thu, Sat}
                      [0, 1, 0, 1, 1, 1]  #accelerated {Tue, Thu, Fri, Sat}
                    ]


""" just simple calculations to see feasibility,
Blocks Demand for senior per month= 3*4=12, junior= 4*4=16, accel=2*7=14 """

#defining total lab hours(actually the no. of blocks) required for each type of groups
Seniors_lab_hours= 3
juniors_lab_hours= 4          # its blocks not hours, where every block is 4 hours
accelerated_lab_hours= 7

#defining no of groups for each type of group
no_of_senior_group= 4
no_of_junior_group= 4
no_of_accelerated_group= 2

days= ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

#Lets define Decision variables, Senior(Xgwdb), junior(Ygwdb), accelerated(Zgwdb) (separated to make the looping easier)
assign_senior= {}
for group in range(no_of_senior_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        assign_senior[(group,week,day,block)]= model.NewBoolVar("Senior_group{}_week{}_day{}_block{}".format(group,week,day,block))

assign_junior= {}
for group in range(no_of_junior_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        assign_junior[(group,week,day,block)]= model.NewBoolVar("Junior_group{}_week{}_day{}_block{}".format(group,week,day,block))

assign_accelerated= {}
for group in range(no_of_junior_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        assign_accelerated[(group,week,day,block)]= model.NewBoolVar("Accelerated_group{}_week{}_day{}_block{}".format(group,week,day,block))


In [ ]:
#Lets define Constraints
#Constraint1: Each type of group(senior, junior and accelerated) need to attend defined no. of lab hours during the whole semester
for group in range(no_of_senior_group):
  seniortotalhours= []
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        seniortotalhours.append(assign_senior[(group,week,day,block)]*block_availability[0][day]) # index 0 in block_availability is for senior
  model.Add(sum(seniortotalhours)>= Seniors_lab_hours)

for group in range(no_of_junior_group):
  juniortotalhours= []
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        juniortotalhours.append(assign_junior[(group,week,day,block)]*block_availability[1][day])
  model.Add(sum(juniortotalhours)>= juniors_lab_hours) # may also try greater than equal to later

for group in range(no_of_accelerated_group):
  acceleratedtotalhours= []
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        acceleratedtotalhours.append(assign_accelerated[(group,week,day,block)]*block_availability[2][day])
  model.Add(sum(acceleratedtotalhours)>= accelerated_lab_hours)

#Constraint2: Every group can only be in 1 block per day
for group in range(no_of_senior_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      seniorsinglesection=[]
      for block in range(no_of_blocks):
        seniorsinglesection.append(assign_senior[(group,week,day,block)])
      model.Add(sum(seniorsinglesection)<=1)

for group in range(no_of_junior_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      juniorsinglesection=[]
      for block in range(no_of_blocks):
        juniorsinglesection.append(assign_junior[(group,week,day,block)]) # first mistake was here
      model.Add(sum(juniorsinglesection)<=1)

for group in range(no_of_accelerated_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      acceleratedsinglesection=[]
      for block in range(no_of_blocks):
        acceleratedsinglesection.append(assign_accelerated[(group,week,day,block)]) #2nd mistake was here
      model.Add(sum(acceleratedsinglesection)<=1)

#constraint3: The lab hours(no of blocks) available every day should not exceed
# this part must have issues
for week in range(no_of_weeks):
  for day in range(no_of_days):
    seniorlabHours= []
    juniorlabHours= []
    acceleratedlabHours= []
    for group in range(no_of_senior_group):
      for block in range(no_of_blocks):
        seniorlabHours.append(assign_senior[(group,week,day,block)])
    for group in range(no_of_junior_group):
      for block in range(no_of_blocks):
        juniorlabHours.append(assign_junior[(group,week,day,block)])
    for group in range(no_of_accelerated_group):
      for block in range(no_of_blocks):
        acceleratedlabHours.append(assign_accelerated[(group,week,day,block)])
    model.Add(sum(seniorlabHours)+sum(juniorlabHours)+sum(acceleratedlabHours)<=sum(blocks_arrangement[day]))


In [ ]:
#these 3 lines of code are the object for Solver unlike the object we created earlier for the model
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0         #for now I do not know what it does
solver.parameters.enumerate_all_solutions = True  # Enumerate all solutions. no idea what happens if I change the value to false

# now lets create the callback function
class callback(cp_model.CpSolverSolutionCallback):
  def __init__(self, assign_senior, assign_junior, assign_accelerated, no_of_days, no_of_blocks, no_of_weeks, no_of_senior_group, no_of_junior_group, no_of_accelerated_group, solution_limit, days):
    cp_model.CpSolverSolutionCallback.__init__(self)
    self._assign_senior= assign_senior
    self._assign_junior= assign_junior
    self._assign_accelerated= assign_accelerated
    self._no_of_days= no_of_days
    self._no_of_blocks= no_of_blocks
    self._no_of_weeks= no_of_weeks
    self._no_of_senior_group= no_of_senior_group
    self._no_of_junior_group= no_of_junior_group
    self._no_of_accelerated_group= no_of_accelerated_group
    self._solution_limit= solution_limit
    self._days= days
    self._solution_count= 0

  def on_solution_callback(self): # no idea if the name of this function can be changed, will try later
    self._solution_count+=1
    print("--------------------------------------------------------------------------------------------------------------------------------")
    print("Solution no.: {}".format(self._solution_count))
    print("--------------------------------------------------------------------------------------------------------------------------------")
    for week in range(self._no_of_weeks):
      print("Week{}: ".format(week+1))
      for day in range(self._no_of_days):
        print("\t",end="") # indentation
        print("\033[1m {} \033[0m".format(self._days[day]), end= ": ")  #Texts between \033[1m  and \033[0m  makes the texts bold
        for block in range(self._no_of_blocks):
          for group in range(self._no_of_senior_group):
            if self.Value(self._assign_senior[(group, week, day, block)]):
              print("Senior-Group{}-Block{}".format(group+1,block+1), end= " |")
          for group in range(self._no_of_junior_group):
            if self.Value(self._assign_junior[(group, week, day, block)]):
              print("Junior-Group{}-Block{}".format(group+1,block+1), end= " |")
          for group in range(self._no_of_accelerated_group):
            if self.Value(self._assign_accelerated[(group, week, day, block)]):
              print("Accelerated-Group{}-Block{}".format(group+1,block+1), end= " |")
        print("\n")
    if self._solution_count>= self._solution_limit:
        self.StopSearch()
#-------------------------------------------------------Invoking the Solver------------------------------------------------------

#time to call the class, and solve
solution_limit= 2
mycallback= callback(assign_senior, assign_junior, assign_accelerated, no_of_days, no_of_blocks, no_of_weeks, no_of_senior_group, no_of_junior_group, no_of_accelerated_group, solution_limit, days)
solver.Solve(model, mycallback)



--------------------------------------------------------------------------------------------------------------------------------
Solution no.: 1
--------------------------------------------------------------------------------------------------------------------------------
Week1: 
	 Monday : Senior-Group4-Block4 |

	 Tuesday : Junior-Group3-Block1 |Junior-Group4-Block1 |Accelerated-Group1-Block2 |Accelerated-Group2-Block3 |

	 Wednesday : Senior-Group4-Block1 |Senior-Group3-Block4 |

	 Thursday : Accelerated-Group1-Block1 |

	 Friday : Junior-Group3-Block1 |Junior-Group4-Block1 |Accelerated-Group1-Block3 |Accelerated-Group2-Block4 |

	 Saturday : Junior-Group3-Block4 |Junior-Group4-Block4 |Accelerated-Group1-Block4 |Accelerated-Group2-Block4 |

Week2: 
	 Monday : Senior-Group4-Block2 |

	 Tuesday : Junior-Group3-Block1 |Junior-Group4-Block1 |Junior-Group2-Block2 |Accelerated-Group2-Block2 |

	 Wednesday : Senior-Group2-Block1 |Senior-Group3-Block4 |

	 Thursday : Accelerated-Group2-Blo

2